In [1]:
# import os
# # import sys
# # from glob import glob
# import numpy as np
# import pandas as pd
# # import cv2
# import torch.nn as nn
# import torch.optim as optim
# from PIL import Image
# from sklearn.model_selection import train_test_split
# from tqdm.notebook import tqdm
# # from time import time
# from matplotlib import image
# from torchvision import transforms, datasets, models
# from torchvision import models
# import time
# import torch
# from torch.utils.data import Dataset, DataLoader, random_split
# from tqdm.notebook import tqdm

# # import matplotlib.pyplot as plt
# import seaborn as sns

In [2]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from PIL import Image
from pandas import DataFrame
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import torch.optim as optim

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

from torchsummary import summary


from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
import torchvision.models as models
import torchvision.utils as vision_utils
import timm

import wandb

In [3]:
### Configurations
data_dir = '/opt/ml/input/data/train'
img_dir = f'{data_dir}/images'
df_path = f'{data_dir}/new_train.csv'
gdf_path = f'{data_dir}/gray_train.csv'
model_path = f'{data_dir}/models'
gs_img_dir = f'{data_dir}/gray_scale_images'

In [4]:
# df = pd.read_csv('/opt/ml/input/data/train/new_train.csv')
# df
# '/opt/ml/input/data/train/new_train.csv'

# Model

In [5]:
class BaseDataset(Dataset):

    def __init__(self,path):
        df = pd.read_csv(path)
        self.X = df['abs_path']
        self.y = df['class']

    def set_transform(self, transform):
        
        self.transform = transform

    def __getitem__(self, index):

        label = self.y[index]
        image = Image.open('/opt/ml/'+self.X[index])
        
        image_transform = self.transform(image=np.array(image))['image']
#         image_transform = self.transform(image)
        return image_transform, label

    def __len__(self):
        return len(self.X)

# Transformers

In [6]:
# from albumentations import *
# from albumentations.pytorch import ToTensorV2


# def get_transforms(need=('train', 'val'), img_size=(512, 384), mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246)):
#     """
#     train 혹은 validation의 augmentation 함수를 정의합니다. train은 데이터에 많은 변형을 주어야하지만, validation에는 최소한의 전처리만 주어져야합니다.
    
#     Args:
#         need: 'train', 혹은 'val' 혹은 둘 다에 대한 augmentation 함수를 얻을 건지에 대한 옵션입니다.
#         img_size: Augmentation 이후 얻을 이미지 사이즈입니다.
#         mean: 이미지를 Normalize할 때 사용될 RGB 평균값입니다.
#         std: 이미지를 Normalize할 때 사용될 RGB 표준편차입니다.

#     Returns:
#         transformations: Augmentation 함수들이 저장된 dictionary 입니다. transformations['train']은 train 데이터에 대한 augmentation 함수가 있습니다.
#     """
#     transformations = {}
#     if 'train' in need:
#         transformations['train'] = Compose([
#             Resize(img_size[0], img_size[1], p=1.0),
#             HorizontalFlip(p=0.5),
#             ShiftScaleRotate(p=0.5),
#             HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
#             RandomBrightnessContrast(brightness_limit=(-0.1, 0.1), contrast_limit=(-0.1, 0.1), p=0.5),
#             GaussNoise(p=0.5),
#             Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
#             ToTensorV2(p=1.0),
#         ], p=1.0)
#     if 'val' in need:
#         transformations['val'] = Compose([
#             Resize(img_size[0], img_size[1]),
#             Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
#             ToTensorV2(p=1.0),
#         ], p=1.0)
#     return transformations

In [7]:
import albumentations as A
from albumentations.pytorch import ToTensorV2


def get_transforms(need=('train', 'val'), img_size=(216, 192), mean=(0.548, 0.504, 0.479), std=(0.237, 0.247, 0.246)):
    """
    train 혹은 validation의 augmentation 함수를 정의합니다. train은 데이터에 많은 변형을 주어야하지만, validation에는 최소한의 전처리만 주어져야합니다.
    
    Args:
        need: 'train', 혹은 'val' 혹은 둘 다에 대한 augmentation 함수를 얻을 건지에 대한 옵션입니다.
        img_size: Augmentation 이후 얻을 이미지 사이즈입니다.
        mean: 이미지를 Normalize할 때 사용될 RGB 평균값입니다.
        std: 이미지를 Normalize할 때 사용될 RGB 표준편차입니다.

    Returns:
        transformations: Augmentation 함수들이 저장된 dictionary 입니다. transformations['train']은 train 데이터에 대한 augmentation 함수가 있습니다.
    """
    transformations = {}
    if 'train' in need:
        transformations['train'] = A.Compose([
            A.Resize(img_size[0], img_size[1], p=1.0),
            A.HorizontalFlip(p=0.5),
            A.Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)
    if 'val' in need:
        transformations['val'] = A.Compose([
            A.Resize(img_size[0], img_size[1]),
            A.Normalize(mean=mean, std=std, max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.0)
    return transformations

### 2nd transformer

In [8]:
mean = [0.56019068, 0.52409788, 0.50145447]
std = [0.23318342, 0.24299835, 0.24567397]

In [9]:
# from torchvision import transforms

# transforms_train = transforms.Compose([
# #     transforms.Resize((512, 384)),
#     transforms.Resize((256,192)),
#     transforms.RandomHorizontalFlip(), # data augmentation
#     transforms.ToTensor(),
#     transforms.Normalize(mean, std), # normalization
# ])

# transforms_val = transforms.Compose([
#     transforms.Resize((256,192)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean, std),
# ])

# Dataset

In [10]:
# 정의한 Augmentation 함수와 Dataset 클래스 객체를 생성합니다.
transform = get_transforms(mean=mean, std=std)

dataset = BaseDataset('/opt/ml/input/data/train/new_train.csv')

# train dataset과 validation dataset을 8:2 비율로 나눕니다.
n_val = int(len(dataset) * 0.2)
n_train = len(dataset) - n_val
train_dataset, val_dataset = random_split(dataset, [n_train, n_val])

# 각 dataset에 augmentation 함수를 설정합니다.
train_dataset.dataset.set_transform(transform['train'])
val_dataset.dataset.set_transform(transform['val'])

### second dataset

In [11]:
# dataset = BaseDataset()

# # train dataset과 validation dataset을 8:2 비율로 나눕니다.
# n_val = int(len(dataset) * 0.2)
# n_train = len(dataset) - n_val
# train_dataset, val_dataset = random_split(dataset, [n_train, n_val])

# # 각 dataset에 augmentation 함수를 설정합니다.
# train_dataset.dataset.set_transform(transforms_train)
# val_dataset.dataset.set_transform(transforms_val)

# DataLoader

In [12]:
# training dataloader은 데이터를 섞어주어야 합니다. (shuffle=True)
train_loader = DataLoader(
    train_dataset,
    batch_size=64,
    num_workers=4,
    shuffle=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=64,
    num_workers=4,
    shuffle=False
)

# Convert to GPU

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device object
device

device(type='cuda', index=0)

# Model

In [14]:
model = models.resnet152(pretrained=True).to(device)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 18).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.01)

### 강의 transformer (epoch = 8) and simple transformer (epohc = 7)
### resnet152 (epoch = 7, 11)

In [15]:
num_epochs = 7

for epoch in range(num_epochs):
    """ Training Phase """
    model.train()

    running_loss = 0.
    running_corrects = 0

    # load a batch data of images
    for inputs, labels in tqdm(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        # forward inputs and get output
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        _, preds = torch.max(outputs, 1)
        

        # get loss value and update the network weights

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects / len(train_dataset)
    print('[Train #{}] Loss: {:.4f} Acc: {:.4f}%'.format(epoch, epoch_loss, epoch_acc))

    """ Validation Phase """
    model.eval()

    with torch.no_grad():
        running_loss = 0.
        running_corrects = 0

        for inputs, labels in tqdm(val_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(val_dataset)
        epoch_acc = running_corrects / len(val_dataset) * 100.
        print('[Validation #{}] Loss: {:.4f} Acc: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc, time.time() - start_time))

KeyboardInterrupt: 

In [ ]:
# save the model
torch.save(model.state_dict(), os.path.join(model_path, 'model_resnet152_transform.pth'))

# Test the model

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import Resize, ToTensor, Normalize

In [ ]:
# 테스트 데이터셋 폴더 경로를 지정해주세요.
test_dir = '/opt/ml/input/data/eval'

In [ ]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [ ]:
from torchvision import transforms
import torchvision

# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=mean, std=std),
])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')

### Test with eval dataset

In [ ]:
def imshow(input, title):
    # torch.Tensor => numpy
    input = input.numpy().transpose((1, 2, 0))
    # undo image normalization
    input = std * input + mean
    input = np.clip(input, 0, 1)
    # display images
    plt.imshow(input)
    plt.title(title)
    plt.show()

In [ ]:
# Test gender model
import torchivision
model.eval()
start_time = time.time()
class_names = [0, 1]

with torch.no_grad():
    running_loss = 0.
    running_corrects = 0

    for i, inputs in enumerate(loader):
        inputs = inputs.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        images = torchvision.utils.make_grid(inputs)
        imshow(images.cpu(), title=preds)
            
        if i == 100:
            break

In [ ]:
# combined model
model.eval()
all_predictions= []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        predicted = predicted.cpu().numpy()
        all_predictions.append(predicted)
        print(predicted)

In [ ]:
for i in range(len(all_predictions)):
    all_predictions[i] = int(all_predictions[i])

print(all_predictions[:5])
print(len(all_predictions))

In [ ]:
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission1.csv'), index=False)
print('test inference is done!')